In [1]:
import numpy as np
import tensorflow as tf

In [19]:
# hidden_size = 4
output_size = 2
input_size = 8

def fully_connected_layer(x, output_size, activation=None, name=None):
    name = name if name is not None else 'fc_layer'
    x_shape = x.shape[-1].value
    W = tf.Variable(initial_value=tf.truncated_normal((x_shape, output_size), stddev=1e-1), name=name+'_weights')
    b = tf.Variable(initial_value=tf.zeros((output_size,), dtype=tf.float32), name=name+'_bias')
    
    return activation(tf.add(tf.matmul(x, W), b))

graph = tf.Graph()
with graph.as_default():
    
    cell_state = tf.placeholder(tf.float32, shape=(None, output_size))
    hidden_state = tf.placeholder(tf.float32, shape=(None, output_size))
    input_ = tf.placeholder(tf.float32, shape=(None, input_size))

    x = tf.concat([hidden_state, input_], 1)
    x_size = output_size + input_size
    
    forget = fully_connected_layer(x, output_size, activation=tf.sigmoid, name='forget')
    input_gate = fully_connected_layer(x, output_size, activation=tf.sigmoid, name='input_gate')
    candidate = fully_connected_layer(x, output_size, activation=tf.tanh, name='candidate')
    
    next_cell_state = tf.add(tf.multiply(forget, cell_state), tf.multiply(input_gate, candidate))
    output = fully_connected_layer(x, output_size, activation=tf.sigmoid, name='output')
    next_hidden_state = tf.multiply(output, tf.tanh(next_cell_state))
    
    with tf.Session() as sess:
        feed_dict = {
            cell_state: np.array([[1, 1]]),
            hidden_state: np.array([[1, 1]]),
            input_: np.array([[1, 1, 1, 1, 1, 1, 1, 1]])
        }
        init = tf.global_variables_initializer()
        sess.run(init)
        sess.run([next_cell_state, output, next_hidden_state], feed_dict=feed_dict)
    